In [338]:
import sys
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable


In [339]:
# Install Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Install pyspark
!pip install pyspark

# Set up Spark session
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("SparkSQL") \
    .config("spark.executor.memory", "2g") \
    .config("spark.driver.memory", "2g") \
    .getOrCreate()


In [340]:
# Import packages
from pyspark.sql import SparkSession
import time
import pandas as pd
from pyspark.sql.functions import col
from pyspark.sql.functions import current_date, expr



# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [341]:
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [342]:
# Path to the CSV files in Google Drive
show_data_path = '/content/drive/My Drive/showData.csv'
movie_data_path = '/content/drive/My Drive/movieData.csv'

In [343]:
# Load the CSV files into DataFrames
show_df = pd.read_csv(show_data_path)
movie_df = pd.read_csv(movie_data_path)


In [344]:
# Display the first few rows of showData.csv
show_df.head()

,Unnamed: 0,id,title,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_score
0,0,ts300399,Five Came Back: The Reference Films,1945,TV-MA,51,['documentation'],['US'],1.0,NaN
1,1,ts22164,Monty Python's Flying Circus,1969,TV-14,30,"['comedy', 'european']",['GB'],4.0,8.8
2,2,ts45948,Monty Python's Fliegender Zirkus,1972,TV-MA,43,['comedy'],[],1.0,8.1
3,3,ts20681,Seinfeld,1989,TV-PG,24,['comedy'],['US'],9.0,8.9
4,4,ts22082,Knight Rider,1982,TV-PG,51,"['scifi', 'action', 'crime', 'drama']",['US'],4.0,6.9


In [345]:
# Display the first few rows of movieData.csv
movie_df.head()

,Unnamed: 0,id,title,release_year,age_certification,runtime,genres,production_countries,imdb_score
0,0,tm84618,Taxi Driver,1976,R,114,"['drama', 'crime']",['US'],8.2
1,1,tm154986,Deliverance,1972,R,109,"['drama', 'action', 'thriller', 'european']",['US'],7.7
2,2,tm127384,Monty Python and the Holy Grail,1975,PG,91,"['fantasy', 'action', 'comedy']",['GB'],8.2
3,3,tm120801,The Dirty Dozen,1967,NaN,150,"['war', 'action']","['GB', 'US']",7.7
4,4,tm70993,Life of Brian,1979,R,94,['comedy'],['GB'],8.0


In [346]:
# Replace NaN IMDb scores with None in show_df_cleaned
show_df_cleaned['imdb_score'] = show_df_cleaned['imdb_score'].apply(lambda x: None if pd.isnull(x) else x)

# Replace NaN IMDb scores with None in movie_df_cleaned
movie_df_cleaned['imdb_score'] = movie_df_cleaned['imdb_score'].apply(lambda x: None if pd.isnull(x) else x)


In [347]:
# Convert cleaned Pandas DataFrames to Spark DataFrames
spark_show_df_cleaned = spark.createDataFrame(show_df_cleaned)
spark_movie_df_cleaned = spark.createDataFrame(movie_df_cleaned)

# Create temporary views for the cleaned Spark DataFrames
spark_show_df_cleaned.createOrReplaceTempView("showData_cleaned")
spark_movie_df_cleaned.createOrReplaceTempView("movieData_cleaned")


In [348]:
# Display the schema of the Spark DataFrame
spark_show_df.printSchema()



root
 |-- Unnamed: 0: long (nullable = true)
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- release_year: long (nullable = true)
 |-- age_certification: string (nullable = true)
 |-- runtime: long (nullable = true)
 |-- genres: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- seasons: double (nullable = true)
 |-- imdb_score: double (nullable = true)



In [349]:
spark_movie_df.printSchema()

root
 |-- Unnamed: 0: long (nullable = true)
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- release_year: long (nullable = true)
 |-- age_certification: string (nullable = true)
 |-- runtime: long (nullable = true)
 |-- genres: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- imdb_score: double (nullable = true)



# What are the top-rated TV shows and movies based on IMDb scores?

In [350]:
# Finding top rated tv shows and movies
top_rated = spark.sql("""
    SELECT 'TV Show' AS type, title, imdb_score
    FROM showData_cleaned
    UNION ALL
    SELECT 'Movie' AS type, title, imdb_score
    FROM movieData_cleaned
    ORDER BY imdb_score DESC
    LIMIT 10
""")
top_rated.show()


+-------+--------------------+----------+
|   type|               title|imdb_score|
+-------+--------------------+----------+
|TV Show|            #ABtalks|       9.6|
|TV Show|        Breaking Bad|       9.5|
|TV Show|            Khawatir|       9.5|
|TV Show|          Our Planet|       9.3|
|TV Show|Avatar: The Last ...|       9.3|
|TV Show|          Reply 1988|       9.2|
|  Movie|               Major|       9.1|
|  Movie|Chhota Bheem & Kr...|       9.1|
|TV Show|      The Last Dance|       9.1|
|TV Show|           My Mister|       9.1|
+-------+--------------------+----------+



# Which genres are most common among TV shows and movies?

In [351]:
result = spark.sql("""
    SELECT type, trim('\"' FROM genre) AS genre, COUNT(*) AS count
    FROM
      (SELECT 'TV Show' AS type, explode(split(trim('[]' FROM genres), ', ')) AS genre FROM showData_cleaned
       UNION ALL
       SELECT 'Movie' AS type, explode(split(trim('[]' FROM genres), ', ')) AS genre FROM movieData_cleaned)
    WHERE genre != ""
    GROUP BY type, genre
    ORDER BY count DESC
""")
# Show the result
result.show()


+-------+---------------+-----+
|   type|          genre|count|
+-------+---------------+-----+
|  Movie|        'drama'| 1805|
|  Movie|       'comedy'| 1493|
|TV Show|        'drama'| 1022|
|  Movie|     'thriller'|  792|
|TV Show|       'comedy'|  725|
|  Movie|       'action'|  688|
|  Movie|      'romance'|  683|
|  Movie|'documentation'|  539|
|  Movie|        'crime'|  533|
|TV Show|       'action'|  421|
|TV Show|    'animation'|  401|
|TV Show|     'thriller'|  388|
|TV Show|        'crime'|  376|
|TV Show|        'scifi'|  360|
|  Movie|     'european'|  329|
|  Movie|       'family'|  328|
|TV Show|       'family'|  324|
|TV Show|'documentation'|  320|
|TV Show|      'fantasy'|  315|
|  Movie|      'fantasy'|  304|
+-------+---------------+-----+
only showing top 20 rows



# How does the distribution of IMDb scores differ between TV shows and movies?

In [352]:
from pyspark.sql.functions import col

# Replace NaN IMDb scores with None in showData
showData_cleaned = spark.sql("""
    SELECT * FROM showData WHERE imdb_score IS NOT NULL
""")
showData_cleaned = showData_cleaned.withColumn("imdb_score", when(col("imdb_score").isNull(), None).otherwise(col("imdb_score")))

# Replace NaN IMDb scores with None in movieData
movieData_cleaned = spark.sql("""
    SELECT * FROM movieData WHERE imdb_score IS NOT NULL
""")
movieData_cleaned = movieData_cleaned.withColumn("imdb_score", when(col("imdb_score").isNull(), None).otherwise(col("imdb_score")))

# Calculate the average IMDb score for TV shows and movies
result = spark.sql("""
    SELECT 'TV Show' AS type, AVG(imdb_score) AS avg_imdb_score
    FROM showData_cleaned
    UNION ALL
    SELECT 'Movie' AS type, AVG(imdb_score) AS avg_imdb_score
    FROM movieData_cleaned
""")
result.show()


+-------+------------------+
|   type|    avg_imdb_score|
+-------+------------------+
|TV Show| 6.977926766374413|
|  Movie|6.2467483231262815|
+-------+------------------+



# What is the average runtime of TV shows and movies?

In [353]:
result = spark.sql("""
    SELECT 'TV Show' AS type, AVG(runtime) AS avg_runtime
    FROM showData_cleaned
    WHERE runtime IS NOT NULL
    UNION ALL
    SELECT 'Movie' AS type, AVG(runtime) AS avg_runtime
    FROM movieData_cleaned
    WHERE runtime IS NOT NULL
    GROUP BY type
""")
result.show()


+-------+------------------+
|   type|       avg_runtime|
+-------+------------------+
|TV Show|38.765858690046414|
|  Movie|101.06620005832605|
+-------+------------------+



# How has the annual release pattern of TV shows and movies evolved over the span of 10 years, from 2013 to 2022?

In [354]:
# Query to count the number of TV shows and movies released each year
result = spark.sql("""
    SELECT type, release_year, COUNT(*) AS count
    FROM
      (SELECT 'TV Show' AS type, release_year FROM showData_cleaned
       UNION ALL
       SELECT 'Movie' AS type, release_year FROM movieData_cleaned)
    WHERE release_year IS NOT NULL
    GROUP BY type, release_year
    ORDER BY release_year DESC, type DESC
    LIMIT 20
""")

# Show the result
result.show()


+-------+------------+-----+
|   type|release_year|count|
+-------+------------+-----+
|TV Show|        2022|  156|
|  Movie|        2022|  180|
|TV Show|        2021|  289|
|  Movie|        2021|  429|
|TV Show|        2020|  257|
|  Movie|        2020|  424|
|TV Show|        2019|  283|
|  Movie|        2019|  465|
|TV Show|        2018|  286|
|  Movie|        2018|  446|
|TV Show|        2017|  173|
|  Movie|        2017|  367|
|TV Show|        2016|  130|
|  Movie|        2016|  213|
|TV Show|        2015|   88|
|  Movie|        2015|  124|
|TV Show|        2014|   41|
|  Movie|        2014|  103|
|TV Show|        2013|   35|
|  Movie|        2013|   92|
+-------+------------+-----+



# Which countries produce the most TV shows and movies?


In [355]:
# Query to count the number of TV shows and movies produced by each country
result = spark.sql("""
    SELECT production_country,
           SUM(CASE WHEN type = 'TV Show' THEN 1 ELSE 0 END) AS tv_count,
           SUM(CASE WHEN type = 'Movie' THEN 1 ELSE 0 END) AS movie_count,
           COUNT(*) AS total_count
    FROM
      (SELECT 'TV Show' AS type, explode(split(production_countries, ', ')) AS production_country FROM showData_cleaned
       UNION ALL
       SELECT 'Movie' AS type, explode(split(production_countries, ', ')) AS production_country FROM movieData_cleaned)
    WHERE production_country != "" AND production_country NOT LIKE "%[%" AND production_country NOT LIKE "%]%"
    GROUP BY production_country
    ORDER BY total_count DESC
""")

# Show the result
result.show()


+------------------+--------+-----------+-----------+
|production_country|tv_count|movie_count|total_count|
+------------------+--------+-----------+-----------+
|              'US'|       3|         33|         36|
|              'GB'|       0|         34|         34|
|              'FR'|       2|         27|         29|
|              'DE'|       0|         19|         19|
|              'CA'|       0|         10|         10|
|              'DK'|       0|         10|         10|
|              'NL'|       0|          8|          8|
|              'CH'|       0|          7|          7|
|              'CN'|       0|          6|          6|
|              'ES'|       0|          5|          5|
|              'BE'|       0|          5|          5|
|              'SE'|       0|          5|          5|
|              'MX'|       0|          5|          5|
|              'JP'|       1|          3|          4|
|              'AU'|       1|          3|          4|
|              'PL'|       0

# Which age certification (e.g., TV-MA, PG-13) is most common among TV shows and movies?

In [356]:
result = spark.sql("""
    SELECT age_certification, COUNT(*) AS count
    FROM
      (SELECT age_certification FROM showData_cleaned WHERE age_certification IS NOT NULL AND age_certification != '' AND age_certification != 'NaN'
       UNION ALL
       SELECT age_certification FROM movieData_cleaned WHERE age_certification IS NOT NULL AND age_certification != '' AND age_certification != 'NaN')
    GROUP BY age_certification
    ORDER BY count DESC
""")
result.show()


+-----------------+-----+
|age_certification|count|
+-----------------+-----+
|            TV-MA|  833|
|                R|  530|
|            TV-14|  441|
|            PG-13|  435|
|               PG|  223|
|            TV-PG|  173|
|            TV-Y7|  111|
|                G|   99|
|             TV-Y|   98|
|             TV-G|   75|
|            NC-17|   15|
+-----------------+-----+



# What is the average number of seasons for TV shows in each age certification category?

In [357]:
result = spark.sql("""
    SELECT age_certification, AVG(seasons) AS avg_seasons
    FROM showData_cleaned
    WHERE age_certification IS NOT NULL AND age_certification != 'NaN'
    GROUP BY age_certification
    ORDER BY avg_seasons DESC
""")

result.show()


+-----------------+------------------+
|age_certification|       avg_seasons|
+-----------------+------------------+
|             TV-G|3.5866666666666664|
|            TV-Y7|3.5225225225225225|
|             TV-Y| 3.122448979591837|
|            TV-PG| 2.820809248554913|
|            TV-14| 2.507936507936508|
|            TV-MA|1.7346938775510203|
+-----------------+------------------+



# How does the average IMDb score vary by production country for TV shows and movies?

In [359]:
result = spark.sql("""
    SELECT type, production_country, AVG(imdb_score) AS avg_imdb_score
    FROM
      (SELECT 'TV Show' AS type, explode(split(production_countries, ', ')) AS production_country, imdb_score FROM showData_cleaned
       UNION ALL
       SELECT 'Movie' AS type, explode(split(production_countries, ', ')) AS production_country, imdb_score FROM movieData_cleaned)
    WHERE production_country != "" AND production_country NOT LIKE "%[%" AND production_country NOT LIKE "%]%" AND imdb_score IS NOT NULL
    GROUP BY type, production_country
    ORDER BY avg_imdb_score DESC
""")

result.show()


+-------+------------------+------------------+
|   type|production_country|    avg_imdb_score|
+-------+------------------+------------------+
|  Movie|              'EG'|               8.1|
|  Movie|              'BS'|               8.0|
|  Movie|              'MW'|               7.6|
|  Movie|              'NO'|              7.35|
|  Movie|              'LB'|               7.3|
|  Movie|              'BR'|               7.2|
|  Movie|              'AE'|               7.1|
|  Movie|              'UY'|               7.1|
|  Movie|              'ES'|7.0200000000000005|
|  Movie|              'GH'|               7.0|
|  Movie|              'ZA'| 6.949999999999999|
|  Movie|              'CH'| 6.914285714285714|
|  Movie|              'MX'|              6.88|
|TV Show|              'FR'|              6.85|
|  Movie|              'IS'| 6.833333333333333|
|  Movie|              'SE'|              6.82|
|  Movie|              'IR'|               6.8|
|  Movie|              'JO'|            